In [1]:
import numpy as np
import argparse
import time
import cv2
import os

In [3]:
labelsPath = os.path.join("model","coco.names")
LABELS = open(labelsPath).read().strip().split("\n")

In [5]:
weightsPath = os.path.join("model","yolov3.weights")
configPath = os.path.join("model","yolov3.cfg")

In [6]:
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

[INFO] loading YOLO from disk...


In [7]:
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [8]:
np.random.seed(49)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")

In [18]:
cap=cv2.VideoCapture("traffic-mini.mp4")
pressed=False
W,H=None,None
while True:
    is_frame,frame=cap.read()
    if not is_frame:
        break
    if W is None or H is None:
        (H, W) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    output_layers = net.forward(ln)
    boxes=[]
    confidences=[]
    classes=[]
    
    for output in output_layers:
        for detection in output:
            scores=detection[5:]
            class_id=np.argmax(scores)
            confidence=scores[class_id]
            
            if confidence > 0.75:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classes.append(class_id)
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.75,0.5)
    if len(idxs)>0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in COLORS[classes[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classes[i]],confidences[i])
            cv2.putText(frame, text, (x, y - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2) 
    cv2.imshow("Object Detection YOLOv3",cv2.resize(frame,(1366,768)))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        pressed=True
        cv2.destroyAllWindows()
        break
if not pressed:
    cv2.destroyAllWindows()